In [50]:
import sys
sys.path.append("..")
from Environment.Env import Vertex, Graph, Agent_to_graph_assignment, cost_calculator
from Environment.Build import Generate_Graph
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Graph parameters

In [39]:
V = 8 # number of vertices
dropout_edge_rate = 0.4 # rate of edges to delete from the fully connected graph

# Create an undirected graph

In [126]:
generator = Generate_Graph(graph_type="Random_Sparse_graph", 
                           nb_vertices= V,
                           dropout_edge_rate=dropout_edge_rate)

In [127]:
g, adj_matrix, all_edges = generator.build()

In [128]:
generator = Generate_Graph(graph_type="OW", 
                          nb_vertices= V,
                          dropout_edge_rate=dropout_edge_rate)

In [129]:
g, adj_matrix, all_edges = generator.build()

In [130]:
adj_matrix

array([[ 0.,  7.,  5., 15.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 7.,  0.,  0., 11., 11.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 5.,  0.,  0.,  7.,  0., 11.,  9.,  0.,  0.,  0.,  0.,  0.,  0.],
       [15., 11.,  7.,  0.,  7.,  0.,  7.,  9.,  0.,  0.,  0.,  0.,  0.],
       [ 0., 11.,  0.,  7.,  0.,  0.,  0.,  7.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0., 11.,  0.,  0.,  0.,  9.,  0., 13.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  9.,  7.,  0.,  9.,  0.,  9.,  0.,  3., 13.,  0.,  0.],
       [ 0.,  0.,  0.,  9.,  7.,  0.,  9.,  0.,  0.,  0.,  3.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0., 13.,  0.,  0.,  0.,  9.,  0.,  2.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  9.,  0.,  9., 12., 12.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 13.,  3.,  0.,  9.,  0.,  0.,  2.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2., 12.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0., 12.,  2.,  0.,  0.]])

# Give names to the agents and randomly assign their starts and destinations

In [131]:
agents =["sto1",'adv1',"hyb1"]
assignement = Agent_to_graph_assignment(graph=g,
                                        list_agents_names = agents, 
                                        adj_matrix=adj_matrix)
agents_dicts = assignement.random_assignement()

In [132]:
agents_dicts

[{'name': 'sto1',
  'infos': {'start': 11,
   'destination': 6,
   'arms': [{'path': [11, 8, 5, 2, 0, 1, 3, 4, 7, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 4, 7, 10, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 4, 7, 10, 9, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 4, 7, 10, 12, 9, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 7, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 7, 10, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 7, 10, 9, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 3, 7, 10, 12, 9, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 3, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 3, 7, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 3, 7, 10, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 3, 7, 10, 9, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 3, 7, 10, 12, 9, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 7, 3, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 7, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 7, 10, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4, 7, 10, 9, 6]},
    {'path': [11, 8, 5, 2, 0, 1, 4,

# Randomly choose an arm = succession of edges towards the arrival

Instead of randomly choosing, we will choose arms using the history of the costs using a Bandit algorithm. This is why we need to code the algorithms and adapt the code in `Agent_to_graph_assignment` to change to choice of arms according to the type of the agent

In [133]:
list_arms_pulled_ = [np.random.choice(agents_dicts[i]["infos"]['arms'])["path"] for i in range(len(agents_dicts))]
list_arms_pulled_

[[11, 8, 5, 2, 0, 1, 4, 3, 7, 6], [12, 9, 6, 3, 2, 5], [0, 3, 1]]

# Compute calculations of costs

In [134]:
cc = cost_calculator(list_arms_pulled=list_arms_pulled_, 
                     adj_matrix= adj_matrix)
summary_round, history_costs_edges = cc.return_costs()

In [135]:
summary_round

{0: {'path': [11, 8, 5, 2, 0, 1, 4, 3, 7, 6], 'cost': 74.0},
 1: {'path': [12, 9, 6, 3, 2, 5], 'cost': 40.0},
 2: {'path': [0, 3, 1], 'cost': 26.0}}

In [136]:
history_costs_edges

{(0, 1): [0, 0, 0, 0, 7.0, 0, 0, 0, 0],
 (0, 2): [0, 0, 0, 5.0, 0, 0, 0, 0, 0],
 (0, 3): [15.0, 0, 0, 0, 0, 0, 0, 0, 0],
 (1, 0): [0, 0, 0, 0, 7.0, 0, 0, 0, 0],
 (1, 3): [0, 11.0, 0, 0, 0, 0, 0, 0, 0],
 (1, 4): [0, 0, 0, 0, 0, 11.0, 0, 0, 0],
 (2, 0): [0, 0, 0, 5.0, 0, 0, 0, 0, 0],
 (2, 3): [0, 0, 0, 7.0, 0, 0, 0, 0, 0],
 (2, 5): [0, 0, 11.0, 0, 11.0, 0, 0, 0, 0],
 (2, 6): [0, 0, 0, 0, 0, 0, 0, 0, 0],
 (3, 0): [15.0, 0, 0, 0, 0, 0, 0, 0, 0],
 (3, 1): [0, 11.0, 0, 0, 0, 0, 0, 0, 0],
 (3, 2): [0, 0, 0, 7.0, 0, 0, 0, 0, 0],
 (3, 4): [0, 0, 0, 0, 0, 0, 7.0, 0, 0],
 (3, 6): [0, 0, 7.0, 0, 0, 0, 0, 0, 0],
 (3, 7): [0, 0, 0, 0, 0, 0, 0, 9.0, 0],
 (4, 1): [0, 0, 0, 0, 0, 11.0, 0, 0, 0],
 (4, 3): [0, 0, 0, 0, 0, 0, 7.0, 0, 0],
 (4, 7): [0, 0, 0, 0, 0, 0, 0, 0, 0],
 (5, 2): [0, 0, 11.0, 0, 11.0, 0, 0, 0, 0],
 (5, 6): [0, 0, 0, 0, 0, 0, 0, 0, 0],
 (5, 8): [0, 13.0, 0, 0, 0, 0, 0, 0, 0],
 (6, 2): [0, 0, 0, 0, 0, 0, 0, 0, 0],
 (6, 3): [0, 0, 7.0, 0, 0, 0, 0, 0, 0],
 (6, 5): [0, 0, 0, 0, 0, 0, 0, 0,

# Compute Minimal cost and optimal paths

### Combinatorial approach : 

In [137]:
assignement.get_optimal_paths(combinatorial=True, time_limit=20)

Time depassed 20 seconds, only 53528 combinations where tested
Total time to compute costs:20.00 s
 => The minimal cost is :  45.0
 => The optimal paths are :  [[11, 9, 6], [12, 10, 7, 6, 5], [0, 1]]
